In [1]:
using Gadfly
using DataFrames
using Colors

In [2]:
# E.g.
set_default_plot_size(26cm, 14cm)

In [3]:
theta1(x) = float(x)^2*(1.0-x)^2
theta2(x) = (x >= 0.25 && x <= 0.75) ? 1 : 0

theta2 (generic function with 1 method)

In [4]:
intervall_discretisation(n) = linspace(1.0/n, 1 - 1.0/n, n-1);

In [5]:
function gradient_descent(f0, A, Theta; gamma=0.0, eps=1.0e-6)
    k = 1
    f = f0
    grad = A\(A\f-Theta) + gamma * f
    while (norm(grad) > eps) && (k < 2e4)
        k = k + 1
        grad =  A\(A\f-Theta) + gamma * f
        
        A_inv_grad = A\grad
        grad_dot = dot(grad, grad)
        sigma_exact = grad_dot / (dot(A_inv_grad, A_inv_grad) + gamma * grad_dot)
        
        f = f - sigma_exact * grad
    end
    
    return f, k
end 

gradient_descent (generic function with 1 method)

In [6]:
function waerme(theta, n; eps=1.0e-6, gamma=0.0)
    e = ones(n-1)
    A = Tridiagonal(-e[2:end], 2e, -e[2:end]) * n^2
    Theta = map(theta, intervall_discretisation(n))

    f, k = gradient_descent(e, A, Theta, gamma=gamma, eps=eps)
    b = A\f
    
    return Dict("f" => f, "b" => b, "k" => k, "Theta" => Theta)
end

waerme (generic function with 1 method)

## Compute Values

Berechne für verschiedene $\gamma$ und $n$

In [7]:
gamma_values = [0; [(iseven(x) ? 1.0 : 0.5)*(10.0^(-floor(x / 2))) for x=16:-1:8]];
print(gamma_values)

[0

In [8]:
values_theta1_10 = [waerme(theta1, 10, gamma=x) for x in gamma_values];
values_theta1_100 = [waerme(theta1, 100, gamma=x) for x in gamma_values];
values_theta1_1000 = [waerme(theta1, 1000, gamma=x) for x in gamma_values];
values_theta2_10 = [waerme(theta2, 10, gamma=x) for x in gamma_values];
values_theta2_100 = [waerme(theta2, 100, gamma=x) for x in gamma_values];
values_theta2_1000 = [waerme(theta2, 1000, gamma=x) for x in gamma_values];

# Plotting

In [9]:
set_default_plot_size(26cm, 36cm)

.0,1.0e-8,5.0e-8,1.0e-7,5.0e-7,1.0e-6,4.9999999999999996e-6,9.999999999999999e-6,5.0e-5,0.0001]

In [10]:
function plot_control_gammas(gamma_values ;theta="theta1", n=1000)
    values
    if (theta == "theta1")
        values = n == 1000 ? values_theta1_1000 : ((n == 100) ? values_theta1_100 : values_theta1_10)
    else
        values = n == 1000 ? values_theta2_1000 : ((n == 100) ? values_theta2_100 : values_theta2_10)
    end
    
    m = length(values)
    cmap = colormap("Blues", m+9, mid=0.5)[end:-1:11]
    legendValues = ["Without regularization"; map(x -> "&#x1D6FE "string(x), gamma_values[2:end])]
    legendColors = [colorant"red"; cmap]
    
    layers = [layer(
        x=intervall_discretisation(n),
        y=values[i]["f"],
        Geom.line,
        Theme(default_color=legendColors[i], line_width=0.5mm)
    ) for i=1:m]
    return plot(layers...,
        Guide.manual_color_key("Legend", legendValues, legendColors),
    Guide.title("Optimal control for " * theta * " and n=" * string(n)),
        Guide.YLabel("Optimal control f")
    )
end

plot_control_gammas (generic function with 1 method)

In [18]:
function plot_solution_gammas(gamma_values ;theta="theta1", n=1000)
    values
    if (theta == "theta1")
        values = n == 1000 ? values_theta1_1000 : ((n == 100) ? values_theta1_100 : values_theta1_10)
    else
        values = n == 1000 ? values_theta2_1000 : ((n == 100) ? values_theta2_100 : values_theta2_10)
    end
    
    m = length(values)
    cmap = colormap("Blues", m+9, mid=0.5)[end:-1:11]
    legendValues = ["Required temperatur"; "Without regularization"; map(x -> "&#x1D6FE "string(x), gamma_values[2:end])]
    legendColors = [colorant"green"; colorant"red"; cmap]
    
    layers = 
    [layer(
        x = intervall_discretisation(n),
        y = values[1]["Theta"],
        Geom.point,
        Theme(default_color=colorant"green", highlight_width=0.2mm, default_point_size=0.8mm)
    );
    [layer(
        x=intervall_discretisation(n),
        y=values[i]["b"],
        Geom.line,
        Theme(default_color=legendColors[i+1], line_width=0.5mm)
            ) for i=1:m]]
    return plot(layers...,
        Guide.manual_color_key("Legend", legendValues, legendColors),
    Guide.title("Optimal solution b for " * theta * " and n=" * string(n)),
    Guide.YLabel("State b")
    )
end

plot_solution_gammas (generic function with 1 method)

In [19]:
vstack(
plot_control_gammas(gamma_values, theta="theta1", n=10),
    plot_control_gammas(gamma_values, theta="theta1", n=100),
    plot_control_gammas(gamma_values, theta="theta1", n=1000)
)    

In [20]:
vstack(
plot_solution_gammas(gamma_values, theta="theta1", n=10),
plot_solution_gammas(gamma_values, theta="theta1", n=100),
plot_solution_gammas(gamma_values, theta="theta1", n=1000)
)

In [21]:
vstack(
plot_control_gammas(gamma_values, theta="theta2", n=10),
plot_control_gammas(gamma_values, theta="theta2", n=100),
plot_control_gammas(gamma_values, theta="theta2", n=1000)
)

In [22]:
vstack(
plot_solution_gammas(gamma_values, theta="theta2", n=10),
plot_solution_gammas(gamma_values, theta="theta2", n=100),
plot_solution_gammas(gamma_values, theta="theta2", n=1000)
)

##### code reste

In [23]:
df_theta1_10 = vcat([
    DataFrame(
        x=intervall_discretisation(10),
        f=values_theta1_10[i]["f"],
        b=values_theta1_10[i]["b"],
        Theta=values_theta1_10[i]["Theta"],
        gamma=gamma_values[i] * ones(9)
    ) 
    for i in 1:length(values_theta1_10)
]...)

df_theta1_100 = vcat([
    DataFrame(
        x=intervall_discretisation(100),
        f=values_theta1_100[i]["f"],
        b=values_theta1_100[i]["b"],
        Theta=values_theta1_100[i]["Theta"],
        gamma=gamma_values[i] * ones(99)
    ) 
    for i in 1:length(values_theta1_100)
]...)

df_theta1_1000 = vcat([
    DataFrame(
        x=intervall_discretisation(1000),
        f=values_theta1_1000[i]["f"],
        b=values_theta1_1000[i]["b"],
        Theta=values_theta1_1000[i]["Theta"],
        gamma=gamma_values[i] * ones(999)
    ) 
    for i in 1:length(values_theta1_1000)
]...);